In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import struct
from array import array
from os.path  import join

In [2]:
!curl -L -o ./data.zip  https://www.kaggle.com/api/v1/datasets/download/hojjatk/mnist-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 22.0M  100 22.0M    0     0   389k      0  0:00:58  0:00:58 --:--:-- 1360k  0  0:01:51  0:00:45  0:01:06  588k


In [3]:
!unzip data.zip

Archive:  data.zip
  inflating: t10k-images-idx3-ubyte/t10k-images-idx3-ubyte  
  inflating: t10k-images.idx3-ubyte  
  inflating: t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte  
  inflating: t10k-labels.idx1-ubyte  
  inflating: train-images-idx3-ubyte/train-images-idx3-ubyte  
  inflating: train-images.idx3-ubyte  
  inflating: train-labels-idx1-ubyte/train-labels-idx1-ubyte  
  inflating: train-labels.idx1-ubyte  


In [4]:
import numpy as np
import pandas as pd
import struct
from array import array

class MnistDataloader(object):
    def __init__(self, training_images_filepath, training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath

    def read_images_labels(self, images_filepath, labels_filepath):
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())

        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())
        images = []
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images.append(img.flatten())  # Flatten the image

        return np.array(images), np.array(labels)  # Return as numpy arrays

    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)

        # Shuffle the training data
        data = np.hstack((y_train.reshape(-1, 1), x_train))  # Combine labels and images
        np.random.shuffle(data)  # Shuffle the data

        Y_train = data[:, 0]  # First column is labels
        X_train = data[:, 1:]  # Remaining columns are images
        X_train = X_train / 255.0  # Normalize pixel values

        return (X_train, Y_train), (x_test, y_test)

# Usage example
# dataloader = MnistDataloader('train-images.idx3-ubyte', 'train-labels.idx1-ubyte', 't10k-images.idx3-ubyte', 't10k-labels.idx1-ubyte')
# (X_train, Y_train), (X_test, Y_test) = dataloader.load_data()


In [5]:
%matplotlib inline
import random
#
# Set file paths based on added MNIST Datasets
#
input_path = './'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

#
# Helper function to show a list of images with their relating titles
#
def show_images(images, title_texts):
    cols = 5
    rows = int(len(images)/cols) + 1
    plt.figure(figsize=(30,20))
    index = 1
    for x in zip(images, title_texts):
        image = x[0]
        title_text = x[1]
        plt.subplot(rows, cols, index)
        plt.imshow(image, cmap=plt.cm.gray)
        if (title_text != ''):
            plt.title(title_text, fontsize = 15);
        index += 1

#
# Load MINST dataset
#
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()


In [ ]:
#data = pd.read_csv('train.csv')

In [ ]:
#data = np.array(data)
#m, n = data.shape
#np.random.shuffle(data)

In [ ]:
#data_dev = data[0:1000].T
#Y_dev = data_dev[0]
#X_dev = data_dev[1:n]
#X_dev = X_dev / 255.

In [ ]:
#data_train = data[1000:m].T
#Y_train = data_train[0]
#X_train = data_train[1:n]
#X_train = X_train / 255.
#_,m_train = X_train.shape

In [6]:

Y_train = y_train
X_train = x_train.T
_, m = X_train.shape

In [ ]:
m

60000

In [ ]:
Y_train

array([6, 9, 6, ..., 0, 1, 4], dtype=uint8)

In [7]:
def init_params():
    W1 = np.random.normal(size=(10, 784)) * np.sqrt(1./(784))
    b1 = np.random.normal(size=(10, 1)) * np.sqrt(1./10)
    W2 = np.random.normal(size=(10, 10)) * np.sqrt(1./20)
    b2 = np.random.normal(size=(10, 1)) * np.sqrt(1./(784))
    return W1, b1, W2, b2


In [8]:
def ReLU(Z):
  return np.maximum(Z, 0)

In [9]:
def softmax(Z):
  exp = np.exp(Z - np.max(Z))
  return exp / exp.sum(axis=0)

In [10]:
def forward_prop(W1, b1, W2, b2, X):
  Z1 = W1.dot(X) + b1
  A1 = ReLU(Z1)
  Z2 = W2.dot(A1) + b2
  A2 = softmax(Z2)

  return Z1, A1, Z2, A2

In [11]:
def ReLU_deriv(Z):
  return Z > 0

In [12]:
def one_hot(Y):
  one_hot_Y = np.zeros((Y.size, Y.max()+1))
  one_hot_Y[np.arange(Y.size), Y] = 1
  one_hot_Y = one_hot_Y.T
  return one_hot_Y

In [13]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

In [14]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
  W1 = W1 - alpha * dW1
  b1 = b1 - alpha * db1
  W2 = W2 - alpha * dW2
  b2 = b2 - alpha * db2

  return W1, b1, W2, b2

In [15]:
def get_predictions(A2):
    return np.argmax(A2, 0)


In [16]:
def get_accuracy(predictions, Y):
  return np.sum(predictions == Y) / Y.size

In [31]:
import cv2
import os
from IPython.display import Video, display

def save_weights_frame(W1, input_image, epoch, folder="frames"):
    os.makedirs(folder, exist_ok=True)

    num_neurons = W1.shape[0]  # Число нейронов первого слоя
    input_size = W1.shape[1]   # Размерность входа
    img_size = int(np.sqrt(input_size))  # Определяем размер изображения

    fig, axes = plt.subplots(1, num_neurons, figsize=(10, 2))

    transformed_weights = W1 @ input_image  # Умножаем веса на входное изображение

    for i, ax in enumerate(axes):
        ax.imshow(transformed_weights[i, :].reshape((img_size, img_size)), cmap='gray')
        ax.axis('off')

    plt.savefig(f"{folder}/frame_{epoch:04d}.png")
    plt.close()

def gradient_descent(X, Y, alpha, iterations, X_test):
    W1, b1, W2, b2 = init_params()
    input_image = X_test[:, 0]  # Берем первое изображение из тестовой выборки

    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        save_weights_frame(W1, input_image, i)

        if i % 10 == 0:
            print("Iteration:", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))

    return W1, b1, W2, b2

def create_video(folder="frames", output_video="weights_evolution.mp4", fps=10):
    images = sorted([img for img in os.listdir(folder) if img.endswith(".png")])
    if not images:
        print("No frames found.")
        return

    frame = cv2.imread(os.path.join(folder, images[0]))
    h, w, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

    for img in images:
        frame = cv2.imread(os.path.join(folder, img))
        video.write(frame)

    video.release()
    print("Video saved as", output_video)

In [32]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 500)

Iteration: 0
0.12676666666666667
Iteration: 10
0.34086666666666665
Iteration: 20
0.4647833333333333
Iteration: 30
0.6314333333333333
Iteration: 40
0.7384166666666667
Iteration: 50
0.7720333333333333
Iteration: 60
0.7963333333333333
Iteration: 70
0.8154666666666667
Iteration: 80
0.82965
Iteration: 90
0.8394
Iteration: 100
0.8468333333333333
Iteration: 110
0.85305
Iteration: 120
0.85855
Iteration: 130
0.8629833333333333
Iteration: 140
0.8667166666666667
Iteration: 150
0.8696166666666667
Iteration: 160
0.8716833333333334
Iteration: 170
0.8742833333333333
Iteration: 180
0.8763166666666666
Iteration: 190
0.8782
Iteration: 200
0.8802333333333333
Iteration: 210
0.8820166666666667
Iteration: 220
0.8832333333333333
Iteration: 230
0.8842666666666666
Iteration: 240
0.8859666666666667
Iteration: 250
0.8869833333333333
Iteration: 260
0.888
Iteration: 270
0.889
Iteration: 280
0.8898833333333334
Iteration: 290
0.8909333333333334
Iteration: 300
0.892
Iteration: 310
0.8928
Iteration: 320
0.893466666666

In [35]:
create_video()  # Создание видео
def show_video(video_path="weights_evolution.mp4"):
    display(Video(video_path))
show_video()

Video saved as weights_evolution.mp4


In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(data, labels, W1, b1, W2, b2):
    correct_predictions = 0
    for index, x in enumerate(data):
      prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
      label = Y_train[index]
      if prediction == label:
        correct_predictions += 1
    print(f'{correct_predictions}/{data.shape[0]}')
    return correct_predictions / data.shape[0] * 100


In [ ]:
Y_test = y_test
X_test = x_test

score = test_prediction(X_test, Y_test, W1, b1, W2, b2)
print(score)

9129/10000
91.29
